In [0]:
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.models import Model
from keras.layers import Dense, Embedding, LSTM, Bidirectional,Input
from keras.callbacks import EarlyStopping
from keras.datasets import imdb
from keras import regularizers
#   Supress warning and informational messages
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [0]:
#   Set parameters for data to use
NUM_WORDS = 6000        # the top most n frequent words to consider
SKIP_TOP = 2            # Skip the top most words that are likely (the, and, a)
MAX_REVIEW_LEN = 100  # Max number of words from a review.

In [0]:
#   Load pre-processed sentiment classified review data from IMDB Database
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words = NUM_WORDS,
                                        skip_top=SKIP_TOP)

In [9]:
word2index = imdb.get_word_index()
print(len(word2index))
#print("encoded word sequence:", x_train[3], "class:", y_train[3])

88584


In [10]:
x_train = sequence.pad_sequences(x_train, maxlen = MAX_REVIEW_LEN)
x_test = sequence.pad_sequences(x_test, maxlen = MAX_REVIEW_LEN)
print('x_train.shape:', x_train.shape, 'x_test.shape:', x_test.shape)

x_train.shape: (25000, 100) x_test.shape: (25000, 100)


In [0]:
def sentiment():
  main_input = Input(shape=(MAX_REVIEW_LEN,), dtype='int32', name='main_input')
  emb = Embedding(input_dim=NUM_WORDS, output_dim=64, input_length=MAX_REVIEW_LEN,
                        dropout=0.2, W_regularizer=regularizers.l2(1e-4))(main_input)
  X = Bidirectional(LSTM(64, dropout=0.3, recurrent_dropout=0.3))(emb)
  X = Dense(1, activation='sigmoid')(X)
  model = Model(inputs=[main_input], outputs=X)
  return model

In [18]:
model=sentiment()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: UserWarning: The `dropout` argument is no longer support in `Embedding`. You can apply a `keras.layers.SpatialDropout1D` layer right after the `Embedding` layer to get the same behavior.
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: UserWarning: Update your `Embedding` call to the Keras 2 API: `Embedding(input_dim=6000, output_dim=64, input_length=100, embeddings_regularizer=<keras.reg...)`
  after removing the cwd from sys.path.


In [0]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [22]:
BATCH_SIZE = 24
EPOCHS = 5
cbk_early_stopping = EarlyStopping(monitor='val_acc', patience=2, mode='max')
model.fit(x_train, y_train, BATCH_SIZE, epochs=EPOCHS,validation_data=(x_test, y_test),callbacks=[cbk_early_stopping])

Train on 25000 samples, validate on 25000 samples
Epoch 1/5
25000/25000 [==============================] - 684s 27ms/step - loss: 0.5180 - acc: 0.7568 - val_loss: 0.4566 - val_acc: 0.8072
Epoch 2/5
25000/25000 [==============================] - 693s 28ms/step - loss: 0.4253 - acc: 0.8339 - val_loss: 0.4235 - val_acc: 0.8335
Epoch 3/5
24984/25000 [============================>.] - ETA: 0s - loss: 0.3902 - acc: 0.8591

KeyboardInterrupt: ignored

In [0]:
model.save('sentiment.h5')

In [0]:
from google.colab import files
files.download('sentiment.h5')

In [26]:
score, acc = model.evaluate(x_test, y_test,
                            batch_size=BATCH_SIZE)
print('test score:', score, ' test accuracy:', acc)

25000/25000 [==============================] - 148s 6ms/step
test score: 0.4186890834760666  test accuracy: 0.8380000001144409
